In [1]:
import os
import gzip
import numpy as np
import json
import networkx as nx
import random

from utilities import qubo_to_ising, matrix_to_couplings

In [17]:
Ntot = 300
graph_type = "SK" # "3Reg" or "ER"
Ngraphs = 50

for idx in range(Ngraphs):
    if graph_type == "3Reg":
        G = nx.random_regular_graph(3, Ntot, seed=idx)
        for (u, v) in G.edges:
            G[u][v]['weight'] = 1

    elif graph_type == "ER":
        G = nx.erdos_renyi_graph(Ntot, 0.5, seed=idx)  # Use this to generate different graphs
        for (u, v) in G.edges:
            G[u][v]['weight'] = 1

    elif graph_type == "SK":
        np.random.seed(idx)
        G = nx.complete_graph(Ntot)
        for (u, v) in G.edges:
            G[u][v]['weight'] = int(np.random.choice([-1, 1]))

    # edges = list(G.edges)
    # degrees = [val for (node, val) in G.degree()]
    # print(f"Graph {idx}: min degree = {min(degrees)}, max degree = {max(degrees)}, avg degree = {sum(degrees)/Ntot}")

    # Assign unity edge weights
    # weights = {edge: 1 for edge in edges} 
    # nx.set_edge_attributes(G, weights, "weight")
    weights = {(i, j): G[i][j]['weight'] for (i, j) in G.edges}

    # Create MaxCut QUBO
    Q = np.zeros((Ntot, Ntot))
    for (i, j) in G.edges:
        # Q[i, j] = 2 * weights[(i, j)]
        # Q[i, i] -= weights[(i, j)]
        # Q[j, j] -= weights[(i, j)]
        Q[i, j] = 2 * G[i][j]['weight']
        Q[i, i] -= G[i][j]['weight']
        Q[j, j] -= G[i][j]['weight']

    # Convert to QUBO and Ising couplings
    J, h = matrix_to_couplings(Q)
    J_ising, h_ising, offset = qubo_to_ising(Q)

    # Save graph
    os.makedirs(f"../data/MaxCut/{graph_type}/{Ntot}v/graphs", exist_ok=True)
    with open(f"../data/MaxCut/{graph_type}/{Ntot}v/graphs/graph{idx}.json", "w") as f:
        # edge and weight
        json.dump({
            "N": Ntot, 
            "edges": {f"{k[0]},{k[1]}": v for k, v in weights.items()}
        }, f, indent=4)

    # Save QUBO
    os.makedirs(f"../data/MaxCut/{graph_type}/{Ntot}v/QUBO", exist_ok=True)
    with open(f"../data/MaxCut/{graph_type}/{Ntot}v/QUBO/qubo_graph{idx}.json", "w") as f:
        json.dump({
            "J": {f"{k[0]},{k[1]}": v for k, v in J.items()}, 
            "h": {str(k): v for k, v in h.items()}
        }, f, indent=4)

    # Save Ising
    os.makedirs(f"../data/MaxCut/{graph_type}/{Ntot}v/Ising", exist_ok=True)
    with open(f"../data/MaxCut/{graph_type}/{Ntot}v/Ising/ising_graph{idx}.json", "w") as f:
        json.dump({
            "J": {f"{k[0]},{k[1]}": v for k, v in J_ising.items()}, 
            "h": {str(k): v for k, v in h_ising.items()}, 
            "c": offset
        }, f, indent=4)